<H1>Latent Class Analysis</H1>

<h3>This algorithm performs parameter estimation using expectation maximization to cluster a dataset of binary variables.</h3>

By "latent class" we mean "unspecified class" i.e. the dataset is missing a column specifying the class type of each row. <br>

The estimated parameters of the model consist of two types. <br>
The first set of parameters are percentages (probabilities) that describe what proportion of the data population belongs to each cluster (latent class).<br>
(This is a list with one probability / percentage / proportion per latent class)<br><br>
The second set of parameters are probabilities that each variable column will contain a value of 1 for each latent class. <br>
(This is a table with a row count equal to the number of latent classes and a column count equal to the number of columns in the input dataset)<br><br>
 
 It is assumed the original dataset is made up of patterns that can be clustered.<br>
 The algorithm will find the parameters by iteratively evaluating the dataset until a minimum amount of improvement is measured or until a hard limit of iterations occurs.<br>
 Once complete the estimated parameters are used in the predict method to assign new rows to one of the latent classes or alternatively with predict_proba assign a probability for each latent class for each row.<br><br>
 Because there is noise in the data the true assignment for every row is not possible.<br>
 e.g. a process for latent class A that normally generates a row like 1,1,1,1,1 will rarely<br> instead generate a row like 1,0,0,0,1 and if 1,0,0,0,1 is an exact or near match for a latent class B then that row will be assigned to B.<br>
 This means that the estimated parameters that are learned will not exactly match the true proportions of each class as long as there is uncertainty / variablility in the dataset.


https://www.sciencedirect.com/science/article/pii/S0022202X2031575X
<br><i>
There are two sets of parameters in an LatentClassAnalysis. <br>
The first is the set of inclusion probabilities (or class membership probabilities), that is, <br>
the probability that any random case in a population will be included in any LC. <br>
<br>
The second parameter is the conditional probability that, given a specific class, a variable takes a certain value.<br>
</i>

![image-caption](LCA.jpg)

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math
from numpy.random import Generator, PCG64
import scipy.stats as stats

In [2]:
class LatentClassAnalysis:
    """
    This class uses expectation maximization to estimate parameters for predicting the proportion of rows
    in the dataset that belong to each latent class and the probability that each column in the dataset will
    have a value of 1 for each of the latent classes.

    The number of latent classes can be specified using the configuration parameter n_classes or if n_classes
    is not specified then class_count_method, diff_percentile_threshold, and least_probable_threshold can be
    set to cause the n_classes to be determined automatically using the method "get_classes_by_prevalence".

    The learning_rate_threshold parameter can be set to cause the iterations to end early if the learning rate
    slows to this threshold as measured by the difference between the summed change in the estimated parameters.
    max_iterations acts as the hard cut off for iterations even if the learning rate threshold has not been hit.

    verbose_level controls the amount of detail in logging.
    random_state is passed into all internal pseudo-randomization methods for reproducibility.

    """
    def __init__(self, init_data=None, n_classes=0, learning_rate_threshold=0.001, max_iterations=100,
                 class_count_method='average_class_count', diff_percentile_threshold=99,
                 least_probable_threshold=1, verbose_level=0, random_state=None):
        self.n_classes = n_classes
        self.class_count_method = class_count_method
        self.learning_rate_threshold = learning_rate_threshold
        self.learning_rate_threshold_hit = False
        self.max_iterations = max_iterations
        self.max_iterations_hit = False
        self.diff_percentile_threshold = diff_percentile_threshold
        self.least_probable_threshold = least_probable_threshold
        self.verbose_level = verbose_level
        self.random_state = random_state


        self.iteration_count = 0

        # model parameters
        self.change_scores = [-np.inf]
        # A list of proportions for each latent class from the entire population
        self.population_to_class_probability = None
        # Each row is a latent class and each column is a variable column from the dataset
        self.variable_to_class_probability = None

        self.classes_by_prevalence = None
        self.median_class_count = 0
        self.average_class_count = 0
        self.expectations = None

    def lookup_prob_for_each_variable_value_to_class(self, data, C):
        # Each 1 or 0 has a probability value
        # variable_to_class_probability defines the probability of a 1 in each variable position
        # stats.bernoulli.pmf gives the probability in variable_to_class_probability if the value is a 1
        # stats.bernoulli.pmf gives the 1 - probability in variable_to_class_probability if the value is a 0
        return stats.bernoulli.pmf(data, p=self.variable_to_class_probability[C])

    def calc_expectation(self, data):
        """
        Calculates the newest probability that each row belongs to each class.
        *** This is equivalent to Mixture Model EM where during each iteration each data row is assigned to a distribution it
        is most likely to belong to so that distribution can get its summary statistics (e.g. mean and stdev)
        recalculated with the rows assigned to it.
        Here instead of assigning rows to a class we just store the probability for each class that each row belongs to it. ***
        :param data: The population of data rows to learn from
        :return: One row for each data row where each column is the probability that the data row belongs to a latent class.
        """
        row_to_class_probabilities = np.zeros(shape=(data.shape[0], self.n_classes))

        for C in range(self.n_classes):
            # Calc the joint probability (multiply the probabilities together) for each row
            per_row_variable_to_class_joint_probability = np.prod(self.lookup_prob_for_each_variable_value_to_class(data, C), axis=1)
            # For each data row multiply the probability for Any row in the population to belong to class C by the
            # joint probability of each variable column value to belong to class C
            per_row_proba_for_class = self.population_to_class_probability[C] * per_row_variable_to_class_joint_probability
            # Persist the probability that each row belongs to each Class
            row_to_class_probabilities[:, C] = per_row_proba_for_class
        # Normalize the probabilities (so they sum to 1.0) across Classes for each row by dividing each Class column probability
        # by the sum total of all Class columns for each row
        normalization_value = np.sum(row_to_class_probabilities, axis=1)
        # Repeat and reshape
        normalization_value = np.tile(normalization_value, (self.n_classes, 1)).T

        return row_to_class_probabilities / normalization_value

    def calc_and_cache_expectations(self, data):

        self.expectations = self.calc_expectation(data)

    def calc_maximization(self, data):


        # Overwrite the population_to_class_probability values with the normalized calculated expectation probabilities
        for C in range(self.n_classes):
            self.population_to_class_probability[C] = np.sum(self.expectations[:, C]) / float(data.shape[0])

        # Overwrite the variable_to_class_probability values with the normalized calculated expectation probabilities
        # for each variable column to each Class.
        for C in range(self.n_classes):
            # Take a single column as a series for this one Class.
            # This is the aggregated probability that each row belongs to the Class.
            rows_probability_for_class = self.expectations[:, C]
            # Reshape the series into a table with a single column so it can be multiplied against another table of columns.
            rows_probability_for_class_reshaped = rows_probability_for_class[:, np.newaxis]
            # Multiply the data rows of binary data against the single column of Class probabilities.
            # This is where the "maximization" part occurs.
            # Columns with 1s keep the row's probability of class membership and columns with 0s just get zero values.
            class_probabilities_foreach_variable_column = rows_probability_for_class_reshaped * data
            # Then we sum up all these probabilities where 1s existed in each column across all rows
            sum_total_across_all_rows_foreach_column = np.sum(class_probabilities_foreach_variable_column, axis=0)
            class_proba_forall_rows = np.sum(rows_probability_for_class)
            # This is the aggregated probability that each variable column belongs to the Class across all rows.
            # In variable_to_class_probability each row represents a Class and
            # each value is the probability of the variables for that Class.
            self.variable_to_class_probability[C] = np.minimum(sum_total_across_all_rows_foreach_column / class_proba_forall_rows, 1.0)

    def fit(self, data):

        self.learning_rate_threshold_hit = False
        self.max_iterations_hit = False

        # If no class count was passed in during initialization then scan the data to determine a class count automatically
        if self.n_classes == 0:
            if self.class_count_method:
                self.classes_by_prevalence = self.get_classes_by_prevalence(data)
                if self.class_count_method == 'median_class_count':
                    self.n_classes = self.median_class_count
                elif self.class_count_method == 'average_class_count':
                    self.n_classes = self.average_class_count
                else:
                    self.n_classes = len(self.classes_by_prevalence[self.class_count_method])
            else:
                raise ValueError(
                    'Either initialize with the number of classes or a class_count_method to automatically determine the number of classes.')

        row_count, column_count = np.shape(data)

        # Initialize the population
        rng = Generator(PCG64(self.random_state))
        self.population_to_class_probability = np.array(rng.gamma(0.5, 1, size=self.n_classes))
        # Renormalize
        self.population_to_class_probability = self.population_to_class_probability / self.population_to_class_probability.sum()

        # Initialize variable to class probabilities
        init_class_probs = []
        for c in range(self.n_classes):
            lst = np.array(rng.gamma(0.5, 1, size=column_count))
            # Renormalize
            lst = lst / lst.sum()
            init_class_probs.append(lst.tolist())
        self.variable_to_class_probability = np.array(init_class_probs)

        import time
        times = []

        for i in range(self.max_iterations):
            t0 = time.perf_counter()

            self.iteration_count += 1

            # Calculate expected probability assignments to each Class for each row
            # and cache the resulting probability assignments in self.expectations
            self.calc_and_cache_expectations(data)

            # Update current population and variable probability to class maps
            self.calc_maximization(data)

            # Stop when change in learning has dropped below learning_rate_threshold or hits max_iterations
            current_probabilities = np.zeros(shape=(row_count, self.n_classes))
            # Sum up current probabilities into a score to compare to the previous score for the change rate
            for C in range(self.n_classes):
                variable_to_class_joint_probability = self.lookup_prob_for_each_variable_value_to_class(data, C)
                variable_to_class_joint_probability = np.prod(variable_to_class_joint_probability, axis=1)
                current_probabilities[:, C] = self.population_to_class_probability[C] * variable_to_class_joint_probability

            # Use a log transform to allow learning_rate_threshold to be set somewhere around 1 or less as a logical range
            change_score = np.sum(np.log(np.sum(current_probabilities, axis=1)))

            if np.abs(change_score - self.change_scores[-1]) < self.learning_rate_threshold:
                self.learning_rate_threshold_hit = True
                break
            else:
                self.change_scores.append(change_score)
                if i == self.max_iterations - 1:
                    self.max_iterations_hit = True
                    break

            t1 = time.perf_counter()
            times.append(t1-t0)

        avg = np.array(times).mean()
        total = np.array(times).sum()
        if self.verbose_level >= 1:
            print('\n===================================================== fit run timings')
            print('avg run time per loop: ', avg)
            print('total run time: ', total)

            print('\n===================================================== fit state')
            print('iteration count:', self.iteration_count)
            print('max_iterations_hit:', self.max_iterations_hit)
            print('learning_rate_threshold_hit:', self.learning_rate_threshold_hit)
            print('n_classes:', self.n_classes)



    def predict(self, data):
        return np.argmax(self.predict_proba(data), axis=1)

    def predict_proba(self, data):
        return self.calc_expectation(data)

    def get_classes_by_prevalence(self, data):
        """
        This method gets a count of each unique pattern of 1s and 0s across all rows,
        and then uses a few different methods to estimate the cut off in the sorted list of counts.

        A variable on the class called median_class_count is set that is the median of all the different estimates.

        Then the estimates for each method are returned.

        The different methods used are:

        largest_diff - Find the largest jump in count difference in the sorted count list.
        least_probable_diff - Compute a list of probabilities from the list of differences using a normal distribution and find the least probable difference from previous counts in the sorted list.
        diff_percentile - Take 1st matching index greater than the Xth percentile diff specified in diff_percentile_threshold
        least_probable_percentile - Take 1st matching index of the Xth percentile least diff probability.



        :param data: The original dataset used to get counts for each unique combination of column values
        :return: A dictionary containing a best guess of class count for each method implemented.
        """

        if self.verbose_level > 0:
            print('** Calculating classes by prevalence **')

        # Create a dictionary of variable pattern keys to their count values sorted largest count to smallest
        counter = Counter(map(tuple, data))
        counts = dict(counter)
        counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

        # Transform the dictionary into a list of tuples to iterate over and lookup by indexes
        if self.verbose_level >= 2:
            print('\n===================================================== Class keys and counts')
        lstKeyValueMap = []
        for k, v in counts.items():
            lstKeyValueMap.append((k, v))
            if self.verbose_level >= 2:
                print(k, v)

        # Reverse the list to be smallest to largest
        rev_lstKeyValueMap = list(reversed(lstKeyValueMap))

        # Iterate over the list of tuples (pattern key, count)
        # Start with the first two items to initialize a normal distribution
        # For each new item count value we will get a probability that it belongs to the normal distribution of all the items that came before
        counter = 2
        diff = max(rev_lstKeyValueMap[1][1] - rev_lstKeyValueMap[0][1], 0)

        # First two items have the same average diff
        threshold_diffs = [diff, diff]
        for i, item in enumerate(rev_lstKeyValueMap[1:-2]):
            next = rev_lstKeyValueMap[i + 2][1]

            d = item[1]
            diff = next - d

            threshold_diffs.append(diff)
            counter += 1
        if self.verbose_level >= 3:
            print('\n===================================================== Sorted diff counts between class keys')
            print(sorted(threshold_diffs))

        counter = 2
        summed = (threshold_diffs[0] + threshold_diffs[1])
        mean = summed / counter
        std = np.std([item for item in threshold_diffs[:counter]])
        std = max(std, 1)
        # First two items have 100% probability
        threshold_probs = [1, 1]
        stats_to_print = []
        for i, item in enumerate(threshold_diffs[2:]):
            d = item
            if mean == 0 and std == 0:
                threshold_prob = 1
            else:
                threshold_prob = stats.norm.pdf(d, loc=mean, scale=std)

            if self.verbose_level >= 4:
                stats_to_print.append([d, round(threshold_prob, 8), round(mean, 4), round(std, 4)])

            threshold_probs.append(threshold_prob)
            counter += 1
            # Update the normal distribution parameters to include this new item for the next evaluation loop
            summed = sum([item for item in threshold_diffs[:counter]])
            mean = summed / counter
            std = np.std([item for item in threshold_diffs[:counter]])

        if self.verbose_level >= 4:
            pd.set_option('display.max_columns', len(stats_to_print[0]))
            pd.set_option('display.max_rows', 30)
            print('\n===================================================== least_probable_diff stats')
            df = pd.DataFrame(stats_to_print, columns=['diff', 'probability', 'mean', 'std']).set_index('diff')
            print(df.head(5))
            print(df.tail(15))

        results = {}

        # 1st matching index of largest diff
        threshold_idx = np.argmax(threshold_diffs)
        results['largest_diff'] = rev_lstKeyValueMap[threshold_idx:]

        # Get the index of the item least probable to belong to the normal distribution of all the items that came before
        # All items above this threshold are considered clusters/distribution peaks/classes
        # 1st matching index for least probable diff
        threshold_idx = np.argmin(threshold_probs)
        results['least_probable_diff'] = rev_lstKeyValueMap[threshold_idx:]

        # 1st matching index greater than the Xth percentile diff specified in diff_percentile_threshold
        threshold_idx = np.argmax(threshold_diffs > np.percentile(threshold_diffs, [self.diff_percentile_threshold]))
        results['diff_percentile'] = rev_lstKeyValueMap[threshold_idx:]

        # 1st matching index of the Xth percentile least diff probability
        threshold_idx = np.argmax(threshold_probs < np.percentile(threshold_probs, [self.least_probable_threshold]))
        results['least_probable_percentile'] = rev_lstKeyValueMap[threshold_idx:]

        self.median_class_count = math.ceil(np.median([len(classes) for classes in results.values()]))
        self.average_class_count = math.ceil(np.mean([len(classes) for classes in results.values()]))

        if self.verbose_level >= 4:
            print('\n===================================================== Classes by prevalence')
            for k, v in results.items():
                print(k)
                for class_key in v:
                    print(class_key)

        if self.verbose_level >= 2:
            print('\n===================================================== Class count method stats')
            print('max count diff between class keys:', max(threshold_diffs))
            print('diff_percentile:', self.diff_percentile_threshold, '%')
            print('least_probable_percentile:', self.least_probable_threshold, '%')
            print('median class count:', self.median_class_count)
            print('average class count:', self.average_class_count)
            for k, v in results.items():
                print(k, 'class count:', len(v))


        return results

<h2>Testing the class</h2>

Generate sample data with known starting probabilities which can be used to compare with the estimated parameters determined by the class.

In [3]:
# Probability properties to generate training data
true_variable_to_class_probability = [
    [0.01, 0.40, 0.90, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    [0.04, 0.90, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    [0.90, 0.90, 0.05, 0.90, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    [0.01, 0.02, 0.30, 0.01, 0.01, 0.90, 0.01, 0.02, 0.01, 0.01],
    [0.01, 0.01, 0.01, 0.03, 0.80, 0.01, 0.01, 0.02, 0.01, 0.01],
    [0.01, 0.02, 0.01, 0.01, 0.02, 0.01, 0.90, 0.01, 0.01, 0.01],
    [0.01, 0.01, 0.01, 0.03, 0.90, 0.01, 0.01, 0.90, 0.01, 0.01],
    [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.90, 0.90]
]
true_population_to_class_probability = [0.05, 0.08, 0.22, 0.14, 0.20, 0.14, 0.12, 0.05]
gen_row_count = 12000
seed = 1
variable_count = len(true_variable_to_class_probability[0])

# Generate training data
training_data = []

dfAll = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9])

# Iterate over per class properties
for pop_to_class, var_to_class in zip(true_population_to_class_probability, true_variable_to_class_probability):
    row_count_by_percentage = int(pop_to_class * gen_row_count)
    rows = stats.bernoulli.rvs(p=var_to_class, size=(row_count_by_percentage, variable_count),
                               random_state=seed).tolist()
    df = pd.DataFrame(rows)
    # Get rid of rows that are all zeros
    df = df.loc[~(df == 0).all(axis=1)]
    rows = df.to_numpy().tolist()
    training_data.extend(rows)
    dfAll = dfAll.append(df)

training_data = np.array(training_data)

print(dfAll.info(verbose=True))
print(dfAll.sample(20))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11167 entries, 0 to 599
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       11167 non-null  object
 1   1       11167 non-null  object
 2   2       11167 non-null  object
 3   3       11167 non-null  object
 4   4       11167 non-null  object
 5   5       11167 non-null  object
 6   6       11167 non-null  object
 7   7       11167 non-null  object
 8   8       11167 non-null  object
 9   9       11167 non-null  object
dtypes: object(10)
memory usage: 959.7+ KB
None
      0  1  2  3  4  5  6  7  8  9
1523  0  0  0  0  0  1  0  0  0  0
1467  0  0  0  0  1  0  0  0  0  0
1380  0  0  0  0  0  0  1  0  0  0
2150  1  1  0  1  1  0  0  0  0  0
1034  0  0  0  0  0  1  0  0  0  0
817   0  1  0  0  0  0  0  0  0  0
292   0  0  0  0  1  0  0  1  0  0
183   1  1  0  1  0  0  0  0  0  0
431   0  0  1  0  0  0  0  0  0  0
340   0  0  0  0  0  0  1  0  0  0
951   0  0  0  0  0  0  1  0

<h3>Setup the class and run the fit command to find the estimated parameters from the test dataset.</h3>

In [4]:
# Initialize algorithm
lca = LatentClassAnalysis(n_classes=8, learning_rate_threshold=0.00001,
                          max_iterations=4000, verbose_level=4, random_state=seed)
# Fit model to the training data
lca.fit(training_data)


===================================================== fit run timings
avg run time per loop:  0.22784605804306676
total run time:  216.90944725699956

===================================================== fit state
iteration count: 953
max_iterations_hit: False
learning_rate_threshold_hit: True
n_classes: 8


<h3>Print the results and compared the estimated parameters to the original test data parameters.</h3>

In [5]:
print("\n==============================================================================================")
print('\ntrue_population_to_class_probability from test data')
print(true_population_to_class_probability)

print("\n==============================================================================================")
print('\ntrue_variable_to_class_probability from test data')
pd.set_option('display.width', 320)
pd.set_option('display.max_columns', variable_count)
print(pd.DataFrame(true_variable_to_class_probability))

print("\n==============================================================================================")
print('\npopulation_to_class_probability from model')
print(np.array2string(lca.population_to_class_probability, max_line_width=320, precision=4))
print('\nvariable_to_class_probability from model')
variable_to_class_probability_to_print = []
for lst in lca.variable_to_class_probability:
    variable_to_class_probability_to_print.append([round(val, 4) for val in lst])
pd.set_option('display.width', 320)
pd.set_option('display.max_columns', variable_count)
print(pd.DataFrame(variable_to_class_probability_to_print))

predicted = lca.predict(training_data)
col_names = ['C' + str(c) for c in range(training_data.shape[1])] + ['pred']
df = pd.DataFrame(np.column_stack((training_data, predicted)), columns=col_names)
groups = df.groupby(['pred'])
class_probs = []
for g in groups:
    col_probs = []
    for col in col_names:
        if col == 'pred':
            continue
        percentage_of_rows_this_column_equals_one = g[1][col].sum() / g[1].shape[0]
        col_probs.append(percentage_of_rows_this_column_equals_one)
    class_probs.append((col_probs, g[0]))

print("\n==============================================================================================")
print(
    "From training data For each predicted cluster the percentage of rows for each column in the data that has a 1 value")
values_to_print = []
for cp in class_probs:
    values_to_print.append([round(val, 4) for val in cp[0]])
    # print(cp[1], [round(val,4) for val in cp[0]])
pd.set_option('display.width', 320)
pd.set_option('display.max_columns', variable_count)
print(pd.DataFrame(values_to_print))



true_population_to_class_probability from test data
[0.05, 0.08, 0.22, 0.14, 0.2, 0.14, 0.12, 0.05]


true_variable_to_class_probability from test data
      0     1     2     3     4     5     6     7     8     9
0  0.01  0.40  0.90  0.02  0.01  0.01  0.01  0.01  0.01  0.01
1  0.04  0.90  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01
2  0.90  0.90  0.05  0.90  0.01  0.01  0.01  0.01  0.01  0.01
3  0.01  0.02  0.30  0.01  0.01  0.90  0.01  0.02  0.01  0.01
4  0.01  0.01  0.01  0.03  0.80  0.01  0.01  0.02  0.01  0.01
5  0.01  0.02  0.01  0.01  0.02  0.01  0.90  0.01  0.01  0.01
6  0.01  0.01  0.01  0.03  0.90  0.01  0.01  0.90  0.01  0.01
7  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.90  0.90


population_to_class_probability from model
[0.104  0.1345 0.2364 0.0384 0.0542 0.0489 0.3007 0.0829]

variable_to_class_probability from model
        0       1       2       3       4       5       6       7       8       9
0  0.0204  1.0000  0.2012  0.0452  0.0031  0.0292  0.0184  0.

<h3>Run the test data through again this time without specifying the number of latent classes so the class automatically guesses how many latent classes there are.</h3>

In [6]:
# Initialize algorithm
lca = LatentClassAnalysis(class_count_method='average_class_count', learning_rate_threshold=0.00001,
                          max_iterations=4000, verbose_level=4, random_state=seed)
# Fit model to the training data
lca.fit(training_data)

** Calculating classes by prevalence **

===================================================== Class keys and counts
(0, 0, 0, 0, 1, 0, 0, 0, 0, 0) 1829
(1, 1, 0, 1, 0, 0, 0, 0, 0, 0) 1762
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0) 1367
(0, 0, 0, 0, 1, 0, 0, 1, 0, 0) 1095
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0) 991
(0, 1, 0, 0, 0, 0, 0, 0, 0, 0) 822
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1) 445
(0, 0, 1, 0, 0, 1, 0, 0, 0, 0) 412
(0, 0, 1, 0, 0, 0, 0, 0, 0, 0) 346
(1, 1, 0, 0, 0, 0, 0, 0, 0, 0) 235
(0, 1, 1, 0, 0, 0, 0, 0, 0, 0) 218
(1, 0, 0, 1, 0, 0, 0, 0, 0, 0) 191
(0, 1, 0, 1, 0, 0, 0, 0, 0, 0) 181
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0) 128
(1, 1, 1, 1, 0, 0, 0, 0, 0, 0) 81
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0) 64
(0, 0, 0, 1, 1, 0, 0, 0, 0, 0) 64
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1) 62
(0, 0, 0, 0, 1, 0, 1, 0, 0, 0) 43
(0, 0, 0, 1, 1, 0, 0, 1, 0, 0) 42
(0, 1, 0, 0, 0, 0, 1, 0, 0, 0) 40
(0, 0, 0, 1, 0, 0, 0, 0, 0, 0) 38
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0) 28
(0, 1, 1, 1, 0, 0, 0, 0, 0, 0) 22
(0, 1, 0, 0, 0, 1, 0, 0, 0, 0) 22
(0, 0, 1, 0, 1,

<h3>Print the results and compared the estimated parameters to the original test data parameters.</h3>

In [7]:
print("\n==============================================================================================")
print('\ntrue_population_to_class_probability from test data')
print(true_population_to_class_probability)

print("\n==============================================================================================")
print('\ntrue_variable_to_class_probability from test data')
pd.set_option('display.width', 320)
pd.set_option('display.max_columns', variable_count)
print(pd.DataFrame(true_variable_to_class_probability))

print("\n==============================================================================================")
print('\npopulation_to_class_probability from model')
print(np.array2string(lca.population_to_class_probability, max_line_width=320, precision=4))
print('\nvariable_to_class_probability from model')
variable_to_class_probability_to_print = []
for lst in lca.variable_to_class_probability:
    variable_to_class_probability_to_print.append([round(val, 4) for val in lst])
pd.set_option('display.width', 320)
pd.set_option('display.max_columns', variable_count)
print(pd.DataFrame(variable_to_class_probability_to_print))

predicted = lca.predict(training_data)
col_names = ['C' + str(c) for c in range(training_data.shape[1])] + ['pred']
df = pd.DataFrame(np.column_stack((training_data, predicted)), columns=col_names)
groups = df.groupby(['pred'])
class_probs = []
for g in groups:
    col_probs = []
    for col in col_names:
        if col == 'pred':
            continue
        percentage_of_rows_this_column_equals_one = g[1][col].sum() / g[1].shape[0]
        col_probs.append(percentage_of_rows_this_column_equals_one)
    class_probs.append((col_probs, g[0]))

print("\n==============================================================================================")
print(
    "From training data For each predicted cluster the percentage of rows for each column in the data that has a 1 value")
values_to_print = []
for cp in class_probs:
    values_to_print.append([round(val, 4) for val in cp[0]])
    # print(cp[1], [round(val,4) for val in cp[0]])
pd.set_option('display.width', 320)
pd.set_option('display.max_columns', variable_count)
print(pd.DataFrame(values_to_print))



true_population_to_class_probability from test data
[0.05, 0.08, 0.22, 0.14, 0.2, 0.14, 0.12, 0.05]


true_variable_to_class_probability from test data
      0     1     2     3     4     5     6     7     8     9
0  0.01  0.40  0.90  0.02  0.01  0.01  0.01  0.01  0.01  0.01
1  0.04  0.90  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01
2  0.90  0.90  0.05  0.90  0.01  0.01  0.01  0.01  0.01  0.01
3  0.01  0.02  0.30  0.01  0.01  0.90  0.01  0.02  0.01  0.01
4  0.01  0.01  0.01  0.03  0.80  0.01  0.01  0.02  0.01  0.01
5  0.01  0.02  0.01  0.01  0.02  0.01  0.90  0.01  0.01  0.01
6  0.01  0.01  0.01  0.03  0.90  0.01  0.01  0.90  0.01  0.01
7  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.90  0.90


population_to_class_probability from model
[0.1196 0.0141 0.2888 0.0157 0.261  0.248  0.0527]

variable_to_class_probability from model
        0       1       2       3       4       5       6       7       8       9
0  0.0000  0.0286  0.0073  0.0115  0.0000  0.0000  1.0000  0.0112  0